In [4]:
import os

# Load .env environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
!pip install twilio

     |████████████████████████████████| 480 kB 5.3 MB/s eta 0:00:01
  Created wheel for twilio: filename=twilio-6.58.0-py2.py3-none-any.whl size=1267711 sha256=617e2f543e2b19c4a07b8f9744f879636a98100cade95b6e6738fdc7704ffec1
  Stored in directory: /Users/hero/Library/Caches/pip/wheels/a9/64/44/233579537f0969ce9149fab02e8afd8a15b23d442d51e791c9
Successfully built twilio


In [6]:
from twilio.rest import Client

In [12]:
# Your Account Sid and Auth Token from twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = os.getenv('TWILIO_ACCOUNT_SID')
auth_token = os.getenv('TWILIO_AUTH_TOKEN')
client = Client(account_sid, auth_token)

message = client.messages.create(
                              body='Hello there!',
                              from_='+18479614395',
                              media_url=['https://demo.twilio.com/owl.png'],
                              to='+14124650602‬'
                          )

print(message.sid)

MM0154f6d49e414761bedbeb897574afe7
